In [1]:
import pandas as pd
import numpy as np
import os
import re
import seaborn as sns
from matplotlib import pyplot as plt
import sys

In [3]:
os.getcwd()

'/Users/raunakadvani/Desktop/2023-2024/Fall/MDI/mdi-research/code/data-cleaning'

In [2]:
# import toxic release data

df_2022 = pd.read_csv("../../data/raw/tri/2022_us.csv")
df_2022.shape

/var/folders/37/fsk42jds3255qblrs5r1v99c0000gn/T/ipykernel_64384/3711914290.py:3: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2022 = pd.read_csv("../../data/raw/tri/2022_us.csv")


(78723, 119)

In [3]:
df_2022.head()

,1. YEAR,2. TRIFD,3. FRS ID,4. FACILITY NAME,5. STREET ADDRESS,6. CITY,7. COUNTY,8. ST,9. ZIP,10. BIA,...,110. 8.2 - ENERGY RECOVER ON,111. 8.3 - ENERGY RECOVER OF,112. 8.4 - RECYCLING ON SITE,113. 8.5 - RECYCLING OFF SIT,114. 8.6 - TREATMENT ON SITE,115. 8.7 - TREATMENT OFF SITE,116. PRODUCTION WSTE (8.1-8.7),117. 8.8 - ONE-TIME RELEASE,118. PROD_RATIO_OR_ ACTIVITY,119. 8.9 - PRODUCTION RATIO
0,2022,02764BWCBL2022W,110006501918,PRYSMIAN CABLES & SYSTEMS USA,22 JOSEPH E WARNER BLVD,NORTH DIGHTON,BRISTOL,MA,2764,NaN,...,0.0,0.0,6.35,0.0,0.0,0.0,2320.113,NaN,ACTIVITY,0.57
1,2022,49548PRMLT5252C,110003583097,SCHREIBER FOOD INC.,5252 CLAY AVE,GRAND RAPIDS,KENT,MI,49548,NaN,...,0.0,0.0,0.00,0.0,75585.0,250.0,75835.000,NaN,PRODUCTION,1.10
2,2022,49548PRMLT5252C,110003583097,SCHREIBER FOOD INC.,5252 CLAY AVE,GRAND RAPIDS,KENT,MI,49548,NaN,...,0.0,0.0,0.00,0.0,0.0,67168.8,74632.000,NaN,PRODUCTION,1.10
3,2022,80002RDYMX555SH,110025334038,READY MIXED CONCRETE PLANT 11,5550 SHERIDAN BLVD,ARVADA,ADAMS,CO,80002,NaN,...,0.0,0.0,0.00,0.0,0.0,0.0,0.000,0.0,NaN,0.00
4,2022,8002WBRNNN7271C,110030489068,BRANNAN SAND & GRAVEL- CENTRAL READY MIX,7271 COLORADO BLVD,COMMERCE CITY,ADAMS,CO,80022,NaN,...,0.0,0.0,0.00,0.0,0.0,0.0,0.000,0.0,NaN,0.00


In [43]:
df["18. FEDERAL FACILITY"]

Lead                                                                                 4046
Zinc compounds                                                                       2927
Nickel                                                                               2478
Chromium                                                                             2446
Copper                                                                               2358
                                                                                     ... 
Nabam                                                                                   1
1-Propanaminium, N,N,N-trimethyl-3-[[(undecafluoropentyl)sulfonyl]amino]-, iodide       1
Poly(difluoromethylene), α-fluoro-ω-[2-(phosphonooxy)ethyl]-, diammonium salt           1
Thiosemicarbazide                                                                       1
S-Ethyl dipropylthiocarbamate                                                           1
Name: 34. 

In [4]:

def clean(directory):
    all_dataframes = []

    for year in range(2000, 2023):  # Loop through years from 2000 to 2022
        filepath = os.path.join(directory, f"{year}_us.csv")

        # check if the file exists
        if os.path.exists(filepath):
            df = pd.read_csv(filepath)
            
            # remove unneeded columns
            columns_to_keep = [
                "1. YEAR", "4. FACILITY NAME", "6. CITY", "7. COUNTY", "8. ST", "9. ZIP", 
                "12. LATITUDE", "13. LONGITUDE", "18. FEDERAL FACILITY", "20. INDUSTRY SECTOR", 
                "34. CHEMICAL", "39. CLEAN AIR ACT CHEMICAL", "43. CARCINOGEN", 
                "62. ON-SITE RELEASE TOTAL", "117. 8.8 - ONE-TIME RELEASE", "119. 8.9 - PRODUCTION RATIO"
            ]
            df = df[columns_to_keep]

            # rename columns
            rename_dict = {
                "1. YEAR" : "year",
                "4. FACILITY NAME" : "facility_name",
                "6. CITY" : "city",
                "7. COUNTY" : "county",
                "8. ST" : "state",
                "9. ZIP" : "zip",
                "12. LATITUDE" : "latitude",
                "13. LONGITUDE" : "longitude",
                "18. FEDERAL FACILITY" : "federal_facility",
                "20. INDUSTRY SECTOR" : "industry_sector",
                "34. CHEMICAL" : "chemical",
                "39. CLEAN AIR ACT CHEMICAL" : "clean_air_act_chemical",
                "43. CARCINOGEN" : "carcinogen",
                "62. ON-SITE RELEASE TOTAL" : "on_site_release_total",
                "117. 8.8 - ONE-TIME RELEASE" : "one_time_release",
                "119. 8.9 - PRODUCTION RATIO" : "production_ratio",
            }
            df.rename(columns=rename_dict, inplace=True)

            # drop rows with any NA values
            # df.dropna(inplace=True)
            
            all_dataframes.append(df)

    # concatenate all dataframes into one
    combined_df = pd.concat(all_dataframes, ignore_index=True)

    return combined_df


In [10]:
df = clean("../../data/raw/tri/")

/var/folders/37/fsk42jds3255qblrs5r1v99c0000gn/T/ipykernel_64384/1530083374.py:9: DtypeWarning: Columns (8,15,20,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/37/fsk42jds3255qblrs5r1v99c0000gn/T/ipykernel_64384/1530083374.py:9: DtypeWarning: Columns (8,10,15,20,21,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/37/fsk42jds3255qblrs5r1v99c0000gn/T/ipykernel_64384/1530083374.py:9: DtypeWarning: Columns (8,15,20,22,23,24,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/37/fsk42jds3255qblrs5r1v99c0000gn/T/ipykernel_64384/1530083374.py:9: DtypeWarning: Columns (15,20,22,23,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/37/fsk42jds3255qblrs5r1v99c0000gn/T/ipykernel_64384/1530083374.py:9

In [14]:
df['year'] = pd.to_datetime(df.year, format='%Y')

In [15]:
df.dtypes

year                      datetime64[ns]
facility_name                     object
city                              object
county                            object
state                             object
zip                               object
latitude                         float64
longitude                        float64
federal_facility                  object
industry_sector                   object
chemical                          object
clean_air_act_chemical            object
carcinogen                        object
on_site_release_total            float64
one_time_release                 float64
production_ratio                 float64
dtype: object

In [16]:
df.head(10)

,year,facility_name,city,county,state,zip,latitude,longitude,federal_facility,industry_sector,chemical,clean_air_act_chemical,carcinogen,on_site_release_total,one_time_release,production_ratio
0,2000-01-01,CHICAGO AEROSOL LLC,BRIDGEVIEW,COOK,IL,60455,41.7390,-87.81060,NO,Hazardous Waste,Xylene (mixed isomers),YES,NO,187.0,0.0,1.17
1,2000-01-01,CHICAGO AEROSOL LLC,BRIDGEVIEW,COOK,IL,60455,41.7390,-87.81060,NO,Hazardous Waste,Ethylbenzene,YES,YES,30.0,0.0,1.17
2,2000-01-01,CHICAGO AEROSOL LLC,BRIDGEVIEW,COOK,IL,60455,41.7390,-87.81060,NO,Hazardous Waste,Toluene,YES,NO,664.0,0.0,1.07
3,2000-01-01,CHICAGO AEROSOL LLC,BRIDGEVIEW,COOK,IL,60455,41.7390,-87.81060,NO,Hazardous Waste,Dichloromethane,YES,YES,412.0,0.0,1.00
4,2000-01-01,CHICAGO AEROSOL LLC,BRIDGEVIEW,COOK,IL,60455,41.7390,-87.81060,NO,Hazardous Waste,Copper compounds,NO,NO,0.0,0.0,0.42
5,2000-01-01,CHICAGO AEROSOL LLC,BRIDGEVIEW,COOK,IL,60455,41.7390,-87.81060,NO,Hazardous Waste,Trichloroethylene,YES,YES,155.0,0.0,1.16
6,2000-01-01,CHICAGO AEROSOL LLC,BRIDGEVIEW,COOK,IL,60455,41.7390,-87.81060,NO,Hazardous Waste,n-Hexane,YES,NO,2260.0,0.0,1.08
7,2000-01-01,CHICAGO AEROSOL LLC,BRIDGEVIEW,COOK,IL,60455,41.7390,-87.81060,NO,Hazardous Waste,Methyl isobutyl ketone,YES,YES,0.0,NaN,NaN
8,2000-01-01,BATAVIA TRANSMISSIONS LLC,BATAVIA,CLERMONT,OH,45103,39.0745,-84.12102,NO,Transportation Equipment,Nitrate compounds (water dissociable; reportab...,NO,NO,1.0,NaN,0.79
9,2000-01-01,BATAVIA TRANSMISSIONS LLC,BATAVIA,CLERMONT,OH,45103,39.0745,-84.12102,NO,Transportation Equipment,Sodium nitrite,NO,NO,1.0,NaN,0.79


In [17]:
df.to_csv("../../data/clean/tri.csv")